In [11]:
import sys
sys.path.append('..')

from ml.dataset.corpus import *
from ml.dataset.mappers_preprocess import *
from ml.dataset.mappers_simplify import *
from ml.dataset.pipeline import *
from IPython.core.display import HTML, display

import matplotlib.pyplot as plt
import pandas as pd
import pychord
import music21
music21.environment.set('musescoreDirectPNGPath', '/usr/bin/musescore')

In [12]:
corpus = SongCorpus()

In [13]:
corpus.load_from_file('/home/sharik/code/Innprak/simple_dataset/guitar_forest_dataset.pickle', max_count=100)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [51]:
s = corpus.songs[5].get_music21_repr()

In [ ]:
s.show()

In [57]:
for c in s[1]:
    c.volume.velocity /= 2

In [58]:
s[1][0].volume

<music21.volume.Volume realized=0.08>

In [74]:
s.show('midi')

In [7]:
corpus.pipeline.mappers = [SplitToGcdMapper(what='all')]
corpus.apply_pipeline_to_memory()

HBox(children=(IntProgress(value=0, max=16463), HTML(value='')))

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/sharik/.pyenv/versions/3.6.0/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/sharik/.pyenv/versions/3.6.0/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/home/sharik/.pyenv/versions/3.6.0/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



{"('SplitToGcdMapper', 'gcd')": {4: 1921,
  8: 2844,
  12: 4,
  16: 22723,
  20: 2,
  24: 10,
  32: 2463,
  48: 339,
  64: 1304,
  80: 1,
  96: 258,
  128: 990,
  160: 30,
  192: 22,
  256: 15}}

In [8]:
corpus.save_to_file('/home/sharik/code/Innprak/simple_dataset/guitar_forest_dataset.pickle')

In [9]:
def map_many(iterable, function, *other):
    if other:
        return map_many(map(function, iterable), *other)
    return map(function, iterable)

def apply_many(elem, function, *other):
    if other:
        return apply_many(function(elem), *other)
    return function(elem)


In [10]:
chords_set = set()
cache = dict()
final_dict = dict()

for song in tqdm.tqdm_notebook(corpus.songs):
    for chord in song.chord_track.chords:
        if chord.notes != []:
            if tuple(chord.notes) not in cache:
                s = chord.get_music21_repr().pitchedCommonName
                final_s = apply_many(s,
                                 lambda s: s.split(' ')[0],
                                 lambda s: s[:3],
                                 lambda s: s[:2] if s[2] == '-' else s, 
                                 lambda s: s.replace('-', 'b'))
                final_dict[tuple(chord.notes)] = final_s
                cache[tuple(chord.notes)] = s
                chords_set.add(final_s)

HBox(children=(IntProgress(value=0, max=12356), HTML(value='')))

In [11]:
len(chords_set)

67

In [12]:
learning_songs = []
for song in corpus.songs:
    melody = [0 if not chord.notes else chord.notes[0].number for chord in song.melody_track.chords]
    rhythm = ['' if not chord.notes else final_dict[tuple(chord.notes)] for chord in song.chord_track.chords]
    learning_songs.append([melody, rhythm])

In [13]:
del corpus

In [20]:
def cut_song(song, len_sample=10):
    notes = np.array(song[0])
    chords = np.array(song[1])
    n = len(song[0])
    
    X, y = np.ndarray(shape=(0, len_sample*2)), []
    for i in range(0, n-len_sample):
        X_notes = notes[i:i+len_sample]
        X_chords = chords[i:i+len_sample]
        
        X_cur = np.hstack([X_notes, X_chords])
        y_cur = chords[i+len_sample]
        
        X = np.vstack([X, X_cur])
        y.append(y_cur)
    return X, y

In [21]:
len_sample = 10
X, y = np.ndarray(shape=(0, len_sample*2)), []
for song in tqdm.tqdm_notebook(learning_songs):
#     try:
    if len(song[0]) > len_sample:
        X_, y_ = cut_song(song, len_sample)
        X = np.vstack([X, X_])
        y = np.hstack([y, y_])
#     except ValueError as e:
#         print("Something wrong:", e)
#         #print(song)
        

HBox(children=(IntProgress(value=0, max=12356), HTML(value='')))

KeyboardInterrupt: 

In [16]:
def cut_song2(song, num_tacts, l_min_note, with_chords=True, delay=4):  # 3 arrays: notes, chords, beat
    l_piece = num_tacts * l_min_note
    #print('l_piece', l_piece)
    notes = np.array(song[0])
    chords = np.array(song[1])
    n = len(song[0])
    #print('n', n, len(song[1]))
    n_items = n // l_piece - 1
    #print('n_items', n_items)
    #print('n_items*l_piece', n_items*l_piece)
    X_notes = notes[:n_items * l_piece].reshape((n_items, l_piece))[:, :-delay]
    X_chords = chords[:n_items * l_piece].reshape((n_items, l_piece))[:, :-delay]

    X = np.hstack([X_notes, X_chords])
    y = np.array([chords[l_piece * (i + 1) + 1] for i in range(n_items)])
    return X, y

In [50]:
songs = learning_songs
n_tacts = 2
X, y = np.ndarray(shape=(0, 56)), []
for song in tqdm.tqdm_notebook(songs):
#     try:
    if len(song[0]) > 32:
        X_, y_ = cut_song2(song, n_tacts, 16, with_chords=True)
        X = np.vstack([X, X_])
        y = np.hstack([y, y_])
#     except ValueError as e:
#         print("Something wrong:", e)
#         #print(song)
        

HBox(children=(IntProgress(value=0, max=12356), HTML(value='')))

In [22]:
X.shape, y.shape

((618836, 20), (618836,))

In [23]:
print(X[0], y[0])
print(X.shape, y.shape)
np.save('X_simple_my.npy', X)
np.save('y_simple_my.npy', y)

['34' '34' '34' '34' '34' '34' '36' '36' '36' '36' 'Bb2' 'Bb2' 'Bb2' 'Bb2'
 'Bb2' 'Bb2' 'C3' 'C3' 'C3' 'C3'] C3
(618836, 20) (618836,)
